In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.linear_model import LogisticRegressionCV 

import pickle
import warnings
warnings.filterwarnings('ignore')

In [6]:
from aux_func import evaluate_model

In [2]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")['fatality']
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")['fatality']

# Regresión Logística con penalización L2 (Ridge)

In [3]:
clf = LogisticRegressionCV(cv=15, random_state=0, n_jobs=4, penalty='l2')
clf.fit(xtrain, ytrain)

In [10]:
with open('../models/LR.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [3]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/LR.pickle', 'rb') as f:
    clf = pickle.load(f)

In [5]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)

In [7]:
evaluate_model(ytest,ypred, ypred_proba)

ROC-AUC score of the model: 0.6522631624980015
Accuracy of the model: 0.9847371212053883



C:\Users\atell\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\atell\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\atell\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report: 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    997197
           1       0.00      0.00      0.00     15456

    accuracy                           0.98   1012653
   macro avg       0.49      0.50      0.50   1012653
weighted avg       0.97      0.98      0.98   1012653


Confusion matrix: 
[[997197      0]
 [ 15456      0]]



## Change Threshold

In [ ]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

# Regresión Logística con penalización L1 (Lasso)

In [4]:
clf = LogisticRegressionCV(cv=10, random_state=0, n_jobs=3, penalty='l1', solver='liblinear')
clf.fit(xtrain, ytrain)

In [ ]:
with open('../models/LRlasso.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [ ]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/LRlasso.pickle', 'rb') as f:
    clf = pickle.load(f)

In [ ]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)

In [ ]:
evaluate_model(ytest, ypred, ypred_proba)

## Change threshold

In [3]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

NameError: name 'ypred_proba' is not defined